In [1]:
!pip install geopandas
!pip install mgwr
!pip install numba

In [2]:
import numpy as np
import pandas as pd
import pickle
import os
import multiprocessing
os.environ['USE_PYGEOS'] = '0'
import geopandas as gp
import multiprocessing as mp
from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW

In [47]:
#Load the funda data
funda_data = gp.read_file("data/funda_buy_amsterdam_31-03-2023_full_distances.gpkg")

funda_data = funda_data.loc[funda_data["price"] < 2500000]


# funda_data = pd.DataFrame(funda_data)

# funda_data = gp.GeoDataFrame(pd.get_dummies(funda_data, columns = ["house_type", "building_type", "energy_label", "has_balcony", "has_garden"]))

In [48]:
#show funda data
funda_data


,X,house_id,house_type,building_type,price,price_m2,room,bedroom,bathroom,living_area,...,city,addressline_city,addressline_zip,addresszip,Postcode4,Opp_m2,tram_dist,metro_dist,train_dist,geometry
0,18,42082895,appartement,Resale property,675000,7500.0,3,2,1,90,...,Amsterdam\n,Amsterdam\n,1073 SC,"Tolstraat 60 3, 1073 SC, Amsterdam\n",1073,585996,166.539057,668.095770,1290.089283,POINT (121858.014 484983.965)
1,51,42057103,appartement,Resale property,1295000,6475.0,5,3,0,200,...,Amsterdam\n,Amsterdam\n,1073 XH,"Quellijnstraat 119 C, 1073 XH, Amsterdam\n",1073,585996,208.399329,373.221221,1985.945869,POINT (121381.001 485541.002)
2,79,42038931,appartement,Resale property,385000,7549.0,4,3,0,51,...,Amsterdam\n,Amsterdam\n,1073 KA,"Burgemeester Tellegenstraat 5 II, 1073 KA, Ams...",1073,585996,375.173601,576.663789,1378.133276,POINT (121691.846 484800.990)
3,107,42005049,appartement,Resale property,650000,7738.1,4,3,1,84,...,Amsterdam\n,Amsterdam\n,1073 VJ,"Tweede Jan van der Heijdenstraat 38 2, 1073 VJ...",1073,585996,78.048891,689.699113,1524.552850,POINT (121834.000 485376.000)
4,212,42165208,appartement,Resale property,465000,8157.9,3,2,1,57,...,Amsterdam\n,Amsterdam\n,1073 RH,"Mauvestraat 37 H, 1073 RH, Amsterdam\n",1073,585996,152.028798,778.780657,1085.901160,POINT (122003.041 484808.014)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,524,88454958,appartement,Resale property,289000,3524.4,3,2,1,82,...,Amsterdam\n,Amsterdam\n,1108 GA,"Leksmondhof 273, 1108 GA, Amsterdam\n",1108,2919328,3134.295481,304.495796,1894.684593,POINT (127290.990 480243.999)
1715,527,88460312,appartement,Resale property,335000,3764.0,4,3,1,89,...,Amsterdam\n,Amsterdam\n,1108 CV,"Leusdenhof 57, 1108 CV, Amsterdam\n",1108,2919328,3157.747057,465.786814,1740.296080,POINT (127136.997 480186.005)
1716,1009,42065127,appartement,Resale property,299000,4271.4,3,2,1,70,...,Amsterdam\n,Amsterdam\n,1108 CT,"Leusdenhof 44, 1108 CT, Amsterdam\n",1108,2919328,3169.672664,486.295154,1719.114500,POINT (127121.997 480171.002)
1717,1460,42912098,appartement,Resale property,280000,4058.0,3,2,1,69,...,Amsterdam\n,Amsterdam\n,1108 ER,"Leksmondhof 94, 1108 ER, Amsterdam\n",1108,2919328,3074.023871,166.117549,2034.389126,POINT (127397.001 480335.000)


In [49]:
#create array with the dependent variable
b_y = funda_data['price'].values.reshape((-1,1))

In [58]:
# #create an array with the indepentend variables (order matters for the extraction of params later)
# b_X = funda_data[['room', 'bedroom', 'bathroom', 'living_area',
#                   'house_age', 'bus_dist', 'subway_dist', 'train_dist',
#                   'university_dist', 'school_dist', 'mall_dist', 'supermarket_dist',
#                   'energy_label_A', 'energy_label_B', 'energy_label_C', 'energy_label_D',
#                   'energy_label_E', 'energy_label_F', 'energy_label_G', 'energy_label_na',
#                   'has_balcony_0.0', 'has_balcony_1.0', 'has_garden_0.0', 'has_garden_1.0']].values 

cols = ['room', 'bedroom', 'bathroom', 'living_area',
                  'house_age', 'train_dist', 'metro_dist', 'tram_dist']

b_X = funda_data[cols].values 

In [59]:
#create coordinate tuple for the model
u = funda_data['geometry'].x
v = funda_data['geometry'].y
b_coords = list(zip(u, v))

In [60]:
multiprocessing.cpu_count()

8

In [61]:
#This might be needed to turn off the OpenMP multi-threading
%env OMP_NUM_THREADS = 1

env: OMP_NUM_THREADS=1


In [62]:
#Parrallelization is more favored when you your data are large and/or your machine have many many cores.
#mgwr has soft dependency of numba, please install numba if you need better performance (pip install numba).
n_proc = 4 #two processors
pool = mp.Pool(n_proc) 

In [63]:
%%time
#Run basic GWR in parrallel mode

gwr_selector = Sel_BW(b_coords, b_y, b_X)
gwr_bw = gwr_selector.search(pool = pool) #add pool to Sel_BW.search
print(gwr_bw)
gwr_results = GWR(b_coords, b_y, b_X, gwr_bw).fit(pool = pool)

87.0
CPU times: total: 109 ms
Wall time: 12.6 s


In [64]:
#show summary
gwr_results.summary()

Model type                                                         Gaussian
Number of observations:                                                1657
Number of covariates:                                                     9

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                       57458178267100.648
Log-likelihood:                                                  -22458.308
AIC:                                                              44934.616
AICc:                                                             44936.750
BIC:                                                           57458178254884.414
R2:                                                                   0.764
Adj. R2:                                                              0.763

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- -----

In [ ]:
%%time
#run MGWR in parrallel mode. Note: max_iter_multi needs to be specified

mgwr_selector = Sel_BW(b_coords, b_y, b_X, multi=True)
mgwr_bw = mgwr_selector.search(pool=pool, max_iter_multi=200, criterion = "AICc") #add pool to Sel_BW.search
print(mgwr_bw)
mgwr_results = MGWR(b_coords, b_y, b_X, selector=mgwr_selector).fit(pool=pool)

Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
#show MGWR model summary
mgwr_results.summary()

In [65]:
#recreate R format table

def return_geopackage(mgwr_results,cols, path_name):
    df = gp.GeoDataFrame()
    df['Intercept'] = mgwr_results.params[:,0]
    df['intercept_SE'] = mgwr_results.bse[:,0]
    df['intercept_TV'] = mgwr_results.tvalues[:,0]
    
    df['yhat'] = mgwr_results.predy
    df['residual'] = mgwr_results.y.reshape((-1,1)) - mgwr_results.predy
    
    for i,col in enumerate(cols):
        
        df[col] = mgwr_results.params[:,i+1]

        df[col+'_SE'] = mgwr_results.bse[:,i+1]
        df[col+'_TV'] = mgwr_results.tvalues[:,i+1]
        
    df['geometry'] = funda_data['geometry']
    df.to_file(path_name)

In [66]:
#show the GeoDataFrame and write it to a folder
return_geopackage(gwr_results,cols, 'data/gwr_results_amsterdam.gpkg')

In [87]:
# pickle.dump(gwr_results, open('data/models/gwr_results.pkl', 'wb'))

In [8]:
# gwr_results = pickle.load(open('data/models/gwr_results.pkl', 'rb'))